# BEGAN을 이용한 이미지생성

라이브러리 읽어들이기

In [0]:
import os

import numpy as np
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import losses
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, Activation, Flatten, Dense, UpSampling2D, Reshape, Lambda, Input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing.image import img_to_array, array_to_img

구글 드라이브 연동하기

In [215]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


이미지를 저장하는 함수

In [0]:
def save_imgs(path, imgs, rows, cols):
    """이미지를 타일 형태로 저장
    
    Arguments:
        path (str): 저장할 폴더 경로
        imgs (np.array): 저장할 이미지 리스트
        rows (int): 타일의 세로 크기
        cols (int): 타일의 가로 크기
        
    Returns:
        None
    """
    base_width = imgs.shape[1]
    base_height = imgs.shape[2]
    channels = imgs.shape[3]
    output_shape = (
        base_height*rows,
        base_width*cols,
        channels
    )
    buffer = np.zeros(output_shape)
    for row in range(rows):
        for col in range(cols):
            img = imgs[row*cols + col]
            buffer[
                row*base_height:(row + 1)*base_height,
                col*base_width:(col + 1)*base_width
            ] = img
    array_to_img(buffer).save(path)

이미지 데이터 읽어 들이기

In [265]:
DATA_DIR = '/content/drive/My Drive/data/'

BATCH_SIZE = 16
IMG_SHAPE = (28, 28, 3)

data_gen = ImageDataGenerator(rescale=1/255.)
train_data_generator = data_gen.flow_from_directory(
    directory=DATA_DIR,
    classes=['mnist'],
    class_mode=None,
    batch_size=BATCH_SIZE,
    target_size=IMG_SHAPE[:2]
)

Found 240 images belonging to 1 classes.


Encoder 정의

In [0]:
def build_encoder(input_shape, z_size, n_filters, n_layers):
    """Encoder구축
    
    Arguments:
        input_shape (int): 이미지의 shape
        z_size (int): 특징 공간의 차원 수
        n_filters (int): 파일 수
        
    Returns:
        model (Model): 인코더 모델 
    """
    model = Sequential()
    model.add(
        Conv2D(
            n_filters,
            3,
            activation='elu',
            input_shape=input_shape,
            padding='same'
        )
    )
    model.add(Conv2D(n_filters, 3, padding='same'))
    for i in range(2, n_layers + 1):
        model.add(
            Conv2D(
                i*n_filters,
                3,
                activation='elu',
                padding='same'
            )
        )
        model.add(
                Conv2D(
                i*n_filters,
                3,
                activation='elu',
                strides=2,
                padding='same'
            )
        )
    model.add(Conv2D(n_layers*n_filters, 3, padding='same'))
    model.add(Flatten())
    model.add(Dense(z_size))
    
    return model

생성자/Decoder 정의

In [0]:
def build_decoder(output_shape, z_size, n_filters, n_layers):
    """Decoder 구축
    
    Arguments:
        output_shape (np.array): 이미지 shape
        z_size (int): 특징 공간의 차원 수
        n_filters (int): 파일 수
        n_layers (int): 레이어 수
        
    Returns:
        model (Model): 디코더 모델 
    """
    # UpSampling2D로 몇 배로 확대할지 계산
    scale = 2**(n_layers - 1)
    # 합성곱층의 처음 입력 사이즈를 scale로부터 역산
    fc_shape = (
        output_shape[0]//scale,
        output_shape[1]//scale,
        n_filters
    )
    # 완전연결 계층에서 필요한 사이즈를 역산
    fc_size = fc_shape[0]*fc_shape[1]*fc_shape[2]
    
    model = Sequential()
    # 완전연결 계층
    model.add(Dense(fc_size, input_shape=(z_size,)))
    model.add(Reshape(fc_shape))
    
    # 합성곱층 반복
    for i in range(n_layers - 1):
        model.add(
            Conv2D(
                n_filters,
                3,
                activation='elu',
                padding='same'
            )
        )
        model.add(
            Conv2D(
                n_filters,
                3,
                activation='elu',
                padding='same'
            )
        )
        model.add(UpSampling2D())
        
    # 마지막 층은 UpSampling2D가 불필요
    model.add(
        Conv2D(
            n_filters,
            3,
            activation='elu',
            padding='same'
        )
    )
    model.add(
        Conv2D(
            n_filters,
            3,
            activation='elu',
            padding='same'
        )
    )
    # 출력층에서는 3채널로
    model.add(Conv2D(3, 3, padding='same'))
    
    return model


생성자 정의

In [0]:
def build_generator(img_shape, z_size, n_filters, n_layers):
    decoder = build_decoder(
        img_shape, z_size, n_filters, n_layers
    )
    return decoder

구분자 정의

In [0]:
def build_discriminator(img_shape, z_size, n_filters, n_layers):
    encoder = build_encoder(
        img_shape, z_size, n_filters, n_layers
    )
    decoder = build_decoder(
        img_shape, z_size, n_filters, n_layers
    )
    return keras.models.Sequential((encoder, decoder))

구분자의 학습용 네트워크

In [0]:
def build_discriminator_trainer(discriminator):
    img_shape = discriminator.input_shape[1:]
    real_inputs = Input(img_shape)
    fake_inputs = Input(img_shape)
    real_outputs = discriminator(real_inputs)
    fake_outputs = discriminator(fake_inputs)

    return Model(
        inputs=[real_inputs, fake_inputs],
        outputs=[real_outputs, fake_outputs]
    )

네트워크 구축

In [271]:
n_filters = 64  #  필터 수
n_layers = 3 # 레이어 수
z_size = 32  #  특징 공간의 차원

generator = build_generator(
    IMG_SHAPE, z_size, n_filters, n_layers
)
discriminator = build_discriminator(
    IMG_SHAPE, z_size, n_filters, n_layers
)
discriminator_trainer = build_discriminator_trainer(
    discriminator
)

generator.summary()
# discriminator.layers[1]은 디코더를 나타냄
discriminator.layers[1].summary()

Model: "sequential_56"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 3136)              103488    
_________________________________________________________________
reshape_34 (Reshape)         (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_324 (Conv2D)          (None, 7, 7, 64)          36928     
_________________________________________________________________
conv2d_325 (Conv2D)          (None, 7, 7, 64)          36928     
_________________________________________________________________
up_sampling2d_64 (UpSampling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_326 (Conv2D)          (None, 14, 14, 64)        36928     
_________________________________________________________________
conv2d_327 (Conv2D)          (None, 14, 14, 64)      

손실(loss) 함수 정의

In [0]:
from tensorflow.python.keras.losses import mean_absolute_error

def build_generator_loss(discriminator):
    # discriminator를 사용해서 손실 함수 정의
    def loss(y_true, y_pred):
        # y_true 
        reconst = discriminator(y_pred)
        return mean_absolute_error(
            reconst,
            y_pred
        )
    return loss

generator 컴파일

In [0]:
# 초기 학습률(Generator)
g_lr = 0.0001

generator_loss = build_generator_loss(discriminator)
generator.compile(
    loss=generator_loss,
    optimizer=Adam(g_lr)
)


discriminator 컴파일

In [0]:
# 초기 학습률(Discriminator)
d_lr = 0.0001

# k_var는 수치(일반 변수)
k_var = 0.0
# k : Keras(TensorFlow) Variable
k = K.variable(k_var)
discriminator_trainer.compile(
    loss=[
        mean_absolute_error,
        mean_absolute_error
    ],
    loss_weights=[1., -k],
    optimizer=Adam(d_lr)
)

수렴 판정용 함수 정의

In [0]:
def measure(real_loss, fake_loss, gamma):
    return real_loss + np.abs(gamma*real_loss - fake_loss)

학습 코드

In [0]:
# k의 갱신에 이용할 파라미터
GAMMA = 0.5
kLambda = 0.001

# 반복 수. 100000～1000000 정도로 지정
TOTAL_STEPS = 10000

# 모델과 확인용 생성 이미지를 저장할 폴더

IMG_SAVE_DIR = '/content/drive/My Drive/data/imgs'
# 확인용으로 5x5 개의 이미지를 생성
IMG_SAMPLE_SHAPE = (5, 5)
N_IMG_SAMPLES = np.prod(IMG_SAMPLE_SHAPE)


# 저장할 폴더가 없다면 생성
os.makedirs(IMG_SAVE_DIR, exist_ok=True)

# 샘플이미지용 랜덤 시드
sample_seeds = np.random.uniform(
    -1, 1, (N_IMG_SAMPLES, z_size)
)

history = []
logs = []

for step, batch in enumerate(train_data_generator):
  

    # 임의의 값(noise) 생성
    z_g = np.random.uniform(
        -1, 1, (BATCH_SIZE, z_size)
    )
    z_d = np.random.uniform(
        -1, 1, (BATCH_SIZE, z_size)
    )
    
    # 생성 이미지(구분자의 학습에 이용)
    g_pred = generator.predict(z_d)
    
    # 생성자를 1스텝 학습시킨다
    generator.train_on_batch(z_g, batch)
    # discriminator 1스텝 학습시킨다
    _, real_loss, fake_loss = discriminator_trainer.train_on_batch(
            [batch, g_pred],
            [batch, g_pred]
    )

    # k 를 갱신
    k_var += kLambda*(GAMMA*real_loss - fake_loss)
    K.set_value(k, k_var)
    

    # g_measure 을 계산하기 위한 loss 저장
    history.append({
        'real_loss': real_loss,
        'fake_loss': fake_loss
    })

    # 10번에 1번씩 로그 표시
    if step%10 == 0:
        # 과거 10 번의 measure 의 평균
        measurement = np.mean([
            measure(
                loss['real_loss'],
                loss['fake_loss'],
                GAMMA
            )
            for loss in history[-10:]
        ])
        
        logs.append({
            'k': K.get_value(k),
            'measure': measurement,
            'real_loss': real_loss,
            'fake_loss': fake_loss
        })
        print(logs[-1])

        # 생성된 이미지 저장  
        img_path = '{}/generated_{}.png'.format(
            IMG_SAVE_DIR,
            step
        )
        save_imgs(
            img_path,
            generator.predict(sample_seeds),
            rows=IMG_SAMPLE_SHAPE[0],
            cols=IMG_SAMPLE_SHAPE[1]
        )

{'k': 0.003983175, 'measure': 0.12288692034780979, 'real_loss': 0.092438966, 'fake_loss': 0.015771529}
{'k': 0.004277925, 'measure': 0.11872204076498746, 'real_loss': 0.0946477, 'fake_loss': 0.01480482}
{'k': 0.0045639463, 'measure': 0.11710310382768511, 'real_loss': 0.07971868, 'fake_loss': 0.0154863475}
{'k': 0.0048392536, 'measure': 0.11399490740150213, 'real_loss': 0.08482627, 'fake_loss': 0.014854597}
{'k': 0.005109405, 'measure': 0.11216168738901615, 'real_loss': 0.08408209, 'fake_loss': 0.015749853}
{'k': 0.0053801867, 'measure': 0.11207628389820457, 'real_loss': 0.08743147, 'fake_loss': 0.014991236}
{'k': 0.0056572063, 'measure': 0.11324446573853493, 'real_loss': 0.091282964, 'fake_loss': 0.015172268}
{'k': 0.0058955783, 'measure': 0.10144953848794103, 'real_loss': 0.07915608, 'fake_loss': 0.014460765}
{'k': 0.006164985, 'measure': 0.11083908881992102, 'real_loss': 0.08703304, 'fake_loss': 0.01531306}
{'k': 0.0064162053, 'measure': 0.10465782834216952, 'real_loss': 0.0782559, '